# Notes:

---
## Create json file for predictions:

In [ ]:
%%bigquery pred
SELECT *
FROM ML.PREDICT(MODEL `statmike-mlops.digits.digits_lr`,(
    SELECT *
    FROM `statmike-mlops.digits.digits_prepped`)
  )

In [ ]:
newob = pred.loc[:0,'p0':'p63'].to_dict(orient='records')
newob = {'instances':newob}
newob

In [ ]:
import json
with open('newob.json','w') as f:
    json.dump(newob,f)

---
### KFP Example

https://cloud.google.com/vertex-ai/docs/pipelines/build-pipeline#google-cloud-components

https://github.com/kubeflow/pipelines/tree/master/components/google-cloud

https://codelabs.developers.google.com/vertex-pipelines-intro#4

In [1]:
from google.cloud import aiplatform
from datetime import datetime
import kfp
import kfp.v2.dsl as dsl
from google_cloud_pipeline_components import aiplatform as gcc_aip

In [43]:
# Locations
REGION = 'us-central1'
PROJECT_ID='statmike-mlops'
BUCKET_NAME='gs://{}/digits/model/02c_automl'.format(PROJECT_ID)
TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")
EXPERIMENT_NAME = '02C_AUTOML'
JOB_NAME = EXPERIMENT_NAME+'_'+TIMESTAMP
MODEL_DIR = '{}/{}'.format(BUCKET_NAME, JOB_NAME)
PIPELINE_ROOT = f"{MODEL_DIR}/pipeline_root/"
PARENT = "projects/" + PROJECT_ID + "/locations/" + REGION

# files
PACKAGE = EXPERIMENT_NAME

Give service account roles/storage.objectAdmin permissions: Console > IAM > Select account projectnumber-compute@developer.gserviceaccount.com > edit - give it the role

In [38]:
SERVICE_ACCOUNT = !gcloud config list --format='value(core.account)' 
SERVICE_ACCOUNT = SERVICE_ACCOUNT[0]
SERVICE_ACCOUNT

'691911073727-compute@developer.gserviceaccount.com'

---

In [9]:
aiplatform.init(project=PROJECT_ID, location=REGION)

In [10]:
!rm -rf {PACKAGE}
!mkdir {PACKAGE}

In [11]:
@dsl.component(base_image="python:3.9")
def product_name(text: str) -> str:
    return text

In [12]:
from typing import NamedTuple

@dsl.component(packages_to_install=["emoji"])
def emoji(text: str) -> NamedTuple("Outputs",
    [
        ("emoji_text", str),  # Return parameters
        ("emoji", str),
    ]
):
    import emoji
    emoji_text = text
    emoji_str = emoji.emojize(':' + emoji_text + ':', use_aliases=True)
    print("output one: {}; output_two: {}".format(emoji_text, emoji_str))
    return (emoji_text, emoji_str)

In [13]:
@dsl.component
def build_sentence(product: str, emoji: str, emojitext: str) -> str:
    print("We completed the pipeline, hooray!")
    end_str = product + " is "
    if len(emoji) > 0:
        end_str += emoji
    else:
        end_str += emojitext
    return(end_str)

In [14]:
@kfp.dsl.pipeline(name="hello-world", description="An intro pipeline", pipeline_root=PIPELINE_ROOT)
def intro_pipeline(text: str = "Vertex Pipelines", emoji_str: str = "sparkles"):
    product_task = product_name(text)
    emoji_task = emoji(emoji_str)
    consumer_task = build_sentence(
        product_task.output,
        emoji_task.outputs["emoji"],
        emoji_task.outputs["emoji_text"]
    )

In [15]:
kfp.v2.compiler.Compiler().compile(
    pipeline_func=intro_pipeline, package_path=f"{PACKAGE}/intro_pipeline_job.json"
)

In [16]:
!gsutil cp {PACKAGE}/*.json $PIPELINE_ROOT

Copying file://automl_02c/intro_pipeline_job.json [Content-Type=application/json]...
/ [1 files][  8.0 KiB/  8.0 KiB]                                                
Operation completed over 1 objects/8.0 KiB.                                      


In [17]:
plJob = aiplatform.PipelineJob(
    display_name=JOB_NAME,
    template_path=f"{PIPELINE_ROOT}intro_pipeline_job.json", #can be gs:, move file there first
    pipeline_root=PIPELINE_ROOT
)

In [18]:
response = plJob.run(service_account = SERVICE_ACCOUNT)

INFO:google.cloud.aiplatform.pipeline_jobs:Creating PipelineJob
INFO:google.cloud.aiplatform.pipeline_jobs:PipelineJob created. Resource name: projects/691911073727/locations/us-central1/pipelineJobs/hello-world-20210907225736
INFO:google.cloud.aiplatform.pipeline_jobs:To use this PipelineJob in another session:
INFO:google.cloud.aiplatform.pipeline_jobs:pipeline_job = aiplatform.PipelineJob.get('projects/691911073727/locations/us-central1/pipelineJobs/hello-world-20210907225736')
INFO:google.cloud.aiplatform.pipeline_jobs:View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/hello-world-20210907225736?project=691911073727
INFO:google.cloud.aiplatform.pipeline_jobs:PipelineJob projects/691911073727/locations/us-central1/pipelineJobs/hello-world-20210907225736 current state:
PipelineState.PIPELINE_STATE_RUNNING
INFO:google.cloud.aiplatform.pipeline_jobs:PipelineJob projects/691911073727/locations/us-central1/pipelineJobs/hello-world-202109072

---